In [1]:
import pandas as pd


In [39]:
df = pd.read_csv("/Users/pedrosanhueza/EXOXY/Administrative Documents/Jobs/geoaire/Planilla I-tree Candelaria.csv")

In [38]:
df

,Fecha,"MP2,5 (µg/m³)",Velocidad del viento (m/s),Altura de \nMezcla (m)
0,44562.00000,25.6,"3,0",124.207
1,44562.04167,7.0,"2,6",78.278
2,44562.08333,5.4,"1,8",51.695
3,44562.12500,4.0,"1,4",50.000
4,44562.16667,11.5,"1,2",50.000
...,...,...,...,...
8317,44926.79167,10.1,"3,7",140.479
8318,44926.83333,11.8,"3,2",64.357
8319,44926.87500,12.5,"3,1",77.685
8320,44926.91667,10.8,"2,4",77.483


In [40]:
df[df.columns[2]] = df[df.columns[2]].apply(lambda x: float(x.replace(",","."))).round()

df

,Fecha,"MP2,5 (µg/m³)",Velocidad del viento (m/s),Altura de \nMezcla (m)
0,44562.00000,25.6,3.0,124.207
1,44562.04167,7.0,3.0,78.278
2,44562.08333,5.4,2.0,51.695
3,44562.12500,4.0,1.0,50.000
4,44562.16667,11.5,1.0,50.000
...,...,...,...,...
8317,44926.79167,10.1,4.0,140.479
8318,44926.83333,11.8,3.0,64.357
8319,44926.87500,12.5,3.0,77.685
8320,44926.91667,10.8,2.0,77.483


In [48]:
table1 = {
    'Velocidad del viento (m/s)': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
    'Promedio': [0, 0.03, 0.09, 0.15, 0.17, 0.19, 0.2, 0.56, 0.92, 0.92, 2.11, 2.11, 2.11, 2.11],
    'Mínimo': [0, 0.006, 0.012, 0.018, 0.022, 0.025, 0.029, 0.056, 0.082, 0.082, 0.57, 0.57, 0.57, 0.57],
    'Máximo': [0, 0.042, 0.163, 0.285, 0.349, 0.414, 0.478, 1.506, 2.534, 2.534, 2.534, 2.534, 2.534, 2.534],
    '% Resuspensión': [0, 1.5, 3, 4.5, 6, 7.5, 9, 10, 11, 12, 13, 16, 20, 23]
}


table1 = pd.DataFrame(table1)

table1

,Velocidad del viento (m/s),Promedio,Mínimo,Máximo,% Resuspensión
0,0,0.00,0.000,0.000,0.0
1,1,0.03,0.006,0.042,1.5
2,2,0.09,0.012,0.163,3.0
3,3,0.15,0.018,0.285,4.5
4,4,0.17,0.022,0.349,6.0
5,5,0.19,0.025,0.414,7.5
6,6,0.20,0.029,0.478,9.0
7,7,0.56,0.056,1.506,10.0
8,8,0.92,0.082,2.534,11.0
9,9,0.92,0.082,2.534,12.0


In [51]:
merged_df = pd.merge(df, table1, on='Velocidad del viento (m/s)', how='left')

merged_df

,Fecha,"MP2,5 (µg/m³)",Velocidad del viento (m/s),Altura de \nMezcla (m),Promedio,Mínimo,Máximo,% Resuspensión
0,44562.00000,25.6,3.0,124.207,0.15,0.018,0.285,4.5
1,44562.04167,7.0,3.0,78.278,0.15,0.018,0.285,4.5
2,44562.08333,5.4,2.0,51.695,0.09,0.012,0.163,3.0
3,44562.12500,4.0,1.0,50.000,0.03,0.006,0.042,1.5
4,44562.16667,11.5,1.0,50.000,0.03,0.006,0.042,1.5
...,...,...,...,...,...,...,...,...
8317,44926.79167,10.1,4.0,140.479,0.17,0.022,0.349,6.0
8318,44926.83333,11.8,3.0,64.357,0.15,0.018,0.285,4.5
8319,44926.87500,12.5,3.0,77.685,0.15,0.018,0.285,4.5
8320,44926.91667,10.8,2.0,77.483,0.09,0.012,0.163,3.0
